

3) Greece
===========

Sessions are listed in http://www.hellenicparliament.gr/Praktika/Synedriaseis-Olomeleias?search=on

Problem 1: only 16 sessions listed at a time: need to search by day
Problem 2: some files are in txt (good), others in docx (not so bad), and others in word 95 .doc (terrible). Fortunately there is a tool called antidoc that dumps the text from the doc file. 

The script scrapes 4 years at a time and needs some babysitting. Data from 1994 to 2017. Thousands of sessions.
Gigabytes of data. You have been warned.

In [1]:

import requests
import bs4

current_year = 2015

i = 0
greece_pages = []
for year in range(current_year,current_year+5):
    for month in range(1,13):
        for day in range(1,31,8):
            strday1 = '{:02d}'.format(day)
            strday2 = '{:02d}'.format(day+8)
            strmonth = '{:02d}'.format(month);
            url = 'http://www.hellenicparliament.gr/Praktika/Synedriaseis-Olomeleias?search=on&DateFrom='+strday1+'%2f'+strmonth+'%2f'+str(year)+'&DateTo='+strday2+'%2f'+strmonth+'%2f'+str(year)
            
            #print(url)
            
            page = requests.get(url);

            soup = bs4.BeautifulSoup(page.text,'html.parser')

            for element in soup.findAll('a'):
                try:
                    page = 'http://www.hellenicparliament.gr' + element['href']
                    #if(page[-4:] == 'docx' or page[-3:] == 'txt' or page[-3:] == 'doc'):
                    if(page[-4:] == 'docx' or page[-3:] == 'txt' or page[-3:] == 'doc'):
                        greece_pages.append(page)
                        print(str(i)+" "+page)
                        i = i + 1
                except: 
                    pass


0 http://www.hellenicparliament.gr/UserFiles/a08fc2dd-61a9-4a83-b09a-09f4c564609d/20170303000409.docx
1 http://www.hellenicparliament.gr/UserFiles/a08fc2dd-61a9-4a83-b09a-09f4c564609d/20170302000408.docx
2 http://www.hellenicparliament.gr/UserFiles/a08fc2dd-61a9-4a83-b09a-09f4c564609d/20170301000407.docx
3 http://www.hellenicparliament.gr/UserFiles/a08fc2dd-61a9-4a83-b09a-09f4c564609d/es20170224000406.docx
4 http://www.hellenicparliament.gr/UserFiles/a08fc2dd-61a9-4a83-b09a-09f4c564609d/es20170223000405.docx
5 http://www.hellenicparliament.gr/UserFiles/a08fc2dd-61a9-4a83-b09a-09f4c564609d/es20170222000404.docx
6 http://www.hellenicparliament.gr/UserFiles/a08fc2dd-61a9-4a83-b09a-09f4c564609d/es20170220000403.docx
7 http://www.hellenicparliament.gr/UserFiles/a08fc2dd-61a9-4a83-b09a-09f4c564609d/es20170217000402.docx
8 http://www.hellenicparliament.gr/UserFiles/a08fc2dd-61a9-4a83-b09a-09f4c564609d/es20170216000401.docx
9 http://www.hellenicparliament.gr/UserFiles/a08fc2dd-61a9-4a83-b09a-0

ConnectionError: HTTPConnectionPool(host='www.hellenicparliament.gr', port=80): Max retries exceeded with url: /Praktika/Synedriaseis-Olomeleias?search=on&DateFrom=01%2f05%2f2018&DateTo=09%2f05%2f2018 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x10830b8d0>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [ ]:
#len(greece_pages)
#myset = set(greece_pages)
#greece_pages=list(myset)

In [2]:
import re 
import time
import locale
import docx
import dateutil
import io
import os
from datetime import datetime
#needs antiword for .doc conversion - 'brew install antiword' for osx

locale.setlocale(locale.LC_TIME, "el_GR")

def getDocxText(text):
    file = io.BytesIO(text)
    doc = docx.Document(file)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

def getDocText(text):
    with open('tmp.doc','wb') as f:
        f.write(text)
    os.system('antiword -m UTF-8.txt tmp.doc > tmp.txt')
    with open('tmp.txt', 'rb') as f:
        return f.read().decode('utf-8','ignore')
    return ''

greece_data = []
i = 0
start = time.time()
for page_url in greece_pages:
    #print(page_url)
    try:
        if(page_url[-3:] == 'txt'): #process Txt
            page = requests.get(page_url)
            text = (page.content.decode('utf-8'))
        if(page_url[-4:] == 'docx'): #process docx
            page = requests.get(page_url)
            text = getDocxText(page.content)
        if(page_url[-3:] == 'doc'): #process docx
            page = requests.get(page_url)
            text = getDocText(page.content)

        s = re.search('[0-9].*Ιανου.*\d\d\d\d|[0-9].*Φεβρο.*\d\d\d\d|[0-9].*Μαϊου.*\d\d\d\d|[0-9].*Μαΐου.*\d\d\d\d|[0-9].*Μαρτίου.*\d\d\d\d|[0-9].*Απριλίου.*\d\d\d\d|[0-9].*Μαίου.*\d\d\d\d|[0-9].*Ιουνίου.*\d\d\d\d|[0-9].*Ιουλίου.*\d\d\d\d|[0-9].*Αυγούστου.*\d\d\d\d|[0-9].*Σεπτεμβρίου.*\d\d\d\d|[0-9].*Οκτωβρίου.*\d\d\d\d|[0-9].*Νοεμβρίου.*\d\d\d\d|[0-9].*Δεκεμβρίου.*\d\d\d\d',text).group(0).strip()
        #print(s)

        # Special cases:
        s = s.replace('1η','1')
        s = s.replace('Μαϊου','Μαΐου')
        s = s.replace('Μαίου','Μαΐου')

        datet = datetime.strptime(s, '%d %B %Y')
        date = datet.strftime("%Y-%m-%d")
        print(s + ' = ' + date+ '         http://'+ page_url[-20:])

        greece_data.append({'date':date,'url':page_url,'text':text})

        i = i + 1
        percentage = i*100/len(greece_pages)
        time_elapsed = time.time()-start
        time_left = time_elapsed/percentage*100
        print("{0:.2f}".format(percentage)+"% "+date+ " " + "{0:.2f}".format(time_elapsed) + " s elapsed - " + "{0:.2f}".format(time_left) + " s total")
    except:
        pass

3 Μαρτίου 2017 = 2017-03-03         http:///20170303000409.docx
0.17% 2017-03-03 1.42 s elapsed - 839.59 s total
2 Μαρτίου 2017 = 2017-03-02         http:///20170302000408.docx
0.34% 2017-03-02 3.91 s elapsed - 1158.30 s total
1 Μαρτίου 2017 = 2017-03-01         http:///20170301000407.docx
0.51% 2017-03-01 5.51 s elapsed - 1086.92 s total
24 Φεβρουαρίου 2017 = 2017-02-24         http://s20170224000406.docx
0.68% 2017-02-24 7.63 s elapsed - 1129.38 s total
23 Φεβρουαρίου 2017 = 2017-02-23         http://s20170223000405.docx
0.84% 2017-02-23 10.46 s elapsed - 1237.88 s total
22 Φεβρουαρίου 2017 = 2017-02-22         http://s20170222000404.docx
1.01% 2017-02-22 11.11 s elapsed - 1095.92 s total
20 Φεβρουαρίου 2017 = 2017-02-20         http://s20170220000403.docx
1.18% 2017-02-20 11.64 s elapsed - 984.49 s total
17 Φεβρουαρίου 2017 = 2017-02-17         http://s20170217000402.docx
1.35% 2017-02-17 12.83 s elapsed - 949.10 s total
16 Φεβρουαρίου 2017 = 2017-02-16         http://s2017021600040

In [4]:

import json

with open('parliament-scraper-greece'+str(current_year)+'.json','w') as f:
    json.dump(greece_data,f)
    

Export it in radio-parlamento's format:


In [ ]:
greece_pages

In [6]:


import json

with open('done/parliament-scraper-greece2010_2014.json','r') as f:
    greece_data = json.load(f, encoding="utf-8")

In [7]:
for data in greece_data:
    date = data['date'].replace('/','-').split(' ')[0]
    print(date)

    
    text = data['text']
    with open('EL/EL_'+date+'.txt','wt') as f:
        f.write(data['url'])
        f.write('\n')
        f.write(text)

2010-01-15
2010-01-14
2010-01-13
2010-01-13
2010-01-12
2010-01-11
2010-01-25
2010-01-22
2010-01-21
2010-01-20
2010-01-19
2010-01-18
2010-02-09
2010-02-08
2010-02-05
2010-02-04
2010-02-03
2010-02-02
2010-02-01
2010-02-17
2010-02-16
2010-02-12
2010-02-11
2010-02-10
2010-02-09
2010-02-25
2010-02-24
2010-02-23
2010-02-22
2010-02-19
2010-02-18
2010-02-17
2010-03-09
2010-03-08
2010-03-05
2010-03-04
2010-03-03
2010-03-01
2010-03-17
2010-03-16
2010-03-15
2010-03-12
2010-03-11
2010-03-10
2010-03-09
2010-03-24
2010-03-23
2010-03-22
2010-03-19
2010-03-18
2010-03-18
2010-03-17
2010-04-16
2010-04-15
2010-04-14
2010-04-14
2010-04-13
2010-04-13
2010-04-12
2010-04-23
2010-04-22
2010-04-21
2010-04-21
2010-04-20
2010-04-19
2010-05-07
2010-05-06
2010-05-05
2010-05-05
2010-05-04
2010-05-03
2010-05-17
2010-05-14
2010-05-13
2010-05-12
2010-05-11
2010-05-10
2010-05-25
2010-05-21
2010-05-20
2010-05-19
2010-05-18
2010-05-17
2010-06-09
2010-06-08
2010-06-07
2010-06-04
2010-06-03
2010-06-02
2010-06-01
2010-06-17